# Serverless Inference API

HuggingFace provides a [Serverless Inference API](https://huggingface.co/docs/inference-providers/index) to quickly test and evaluate ML models with simple API calls.

## Setups

In this example, we need a fine-grained token with
- `Inference > Make calls to the serverless Inference API` user permissions,
- read access to `meta-llama/Meta-Llama-3-8B-Instruct` and `HuggingFaceM4/idefics2-8b-chatty` repos

In [ ]:
!pip install -qU huggingface_hub transformers

## Querying the Serverless Inference API

The Serverless Inference API exposes models on the Hub with a simple API:
```
https://api-inference.huggingface.co/models/<MODEL_ID>
```
where `<MODEL_ID>` corresponds to the name of the model repo on the Hub. For example, `codellama/CodeLlama-7b-hf` becomes `https://api-inference.huggingface.co/models/codellama/CodeLlama-7b-hf`.

### With an HTTP request

We can call this API with a simple `POST` request.

In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/codellama/CodeLlama-7b-hf"
HEADERS = {"Authorization": f"Bearer {get_token()}"}


def query(payload):
    response = requests.post(
        API_URL,
        headers=HEADERS,
        json=payload
    )
    return response.json()

In [ ]:
print(query(
    payload={
        'inputs': "A HTTP POST request is used to",
        'parameters': {'temperature': 0.8, 'max_new_tokens': 50, 'seed': 111}
    }
))

The inference API will dynamically load the requested model onto shared compute infrastructure to serve predictions. When the model is loaded, the Serverless Inference API will use the specified `pipeline_tag` from the Model Card to determine the appropriate inference task.

### With the `huggingface_hub` library

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient()

In [ ]:
response = client.text_generation(
    prompt="A HTTP POST request is used to",
    model="codellama/CodeLlama-7b-hf",
    temperature=0.8,
    max_new_tokens=50,
    seed=111,
    return_full_text=True
)

print(response)

## Applications

### Generating text with open LLMs

- **Base models** - refer to plan, pre-trained language models. These models are good at continuing generation from a provided prompt. However, they have not been fine-tuned for conversational use like answering questions.
- **Instruction-tuned models** - trained in a multi-task manner to follow a broad range of instructions. Instruction-tuned models will produce better responses to instructions than base models. Often, these models are also fine-tuned for multi-turn chat dialogs, making them great for conversational use cases.

In [ ]:
from transformers import AutoTokenizer

# define the system and user messages
system_input = "You're an expert prompt engineer with artistic flair."
user_input = "Write a concise prompt for a fun image containing a llama and a cookbook. Only return the prompt."
messages = [
    {'role', 'system', 'content': system_input},
    {'role', 'user', 'content': user_input}
]

# load the tokenizer
model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# apply the chat template to the messages
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(f"\nPROMPT:\n-----\n\n{prompt}")

In [ ]:
llm_response = client.text_generation(
    prompt,
    model=model_id,
    max_new_tokens=250,
    seed=111
)

Querying an LLM without adhering to the model's prompt template will not produce any outright errors but it will result in poor quality outputs.

In [ ]:
out = client.text_generation(
    system_input + " " + user_input,
    model=model_id,
    max_new_tokens=250,
    seed=111
)
print(out)

To simplify the prompting process and ensure the proper chat template is being used, the `InferenceClient` also offers a `chat_completion` method that abstracts away the `chat_template` details:

In [ ]:
for token in client.chat_completion(
    messages,
    model=model_id,
    max_tokens=250,
    stream=True,
    seed=111,
):
    print(token.choices[0].delta.content)

The `stream=True` enables streaming text from the endpoint.

### Creating images with Stable Diffusion

In [ ]:
image = client.text_to_image(
    prompt=llm_response,
    model='stabilityai/stable-diffusion-xl-base-1.0',
    guidance_scale=8,
    seed=111
)

In [ ]:
display(image.resize((image.width // 2, image.height // 2)))
print("PROMPT: ", llm_response)

The `InferenceClient` will cache API responses by default. That means if we query the API with the same payload multiple times, we will see the result returned by the API is exactly the same:

In [ ]:
image = client.text_to_image(
    prompt=llm_response,
    model='stabilityai/stable-diffusion-xl-base-1.0',
    guidance_scale=8,
    seed=111
)

display(image.resize((image.width // 2, image.height // 2)))
print("PROMPT: ", llm_response)

To force a different response each time, we can use a HTTP header to have the client ignore the cache and run a new generation: `x-use-cache: 0`.

In [ ]:
# turn caching off
client.headers['x-use-cache'] = "0"

image = client.text_to_image(
    prompt=llm_response,
    model='stabilityai/stable-diffusion-xl-base-1.0',
    guidance_scale=8,
    seed=111
)

display(image.resize((image.width // 2, image.height // 2)))
print("PROMPT: ", llm_response)

### Reasoning over images with Idefics2

Vision Language Models (VLMs) can take both text and images as input simultaneously and produce text as output. This allows them to tackle many tasks from visual question answering to image captioning.

For images, we first need to convert our PIL image to a `base64` encoded string so that we can send it to the model over the network.

In [ ]:
import base64
from io import BytesIO

def pil_image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format='JPEG')
    img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
    return img_str


img_b64 = pil_image_to_base64(image)

Then we need to properly format our text and image prompt using a chat template.

In [ ]:
from transfomrers import AutoProcessor

# load the processor
vlm_model_id = 'HuggingFaceM4/idefics2-8b-chatty'
processor = AutoProcessor.from_pretrained(vlm_model_id)

# define the user message
messages = [
    {
        'role': 'user',
        'content': [
            {'type': 'image'},
            {'type': 'text', 'text': "Write a short limerick about this image."}
        ]
    }
]

# apply the chat template to the messages
prompt = processor.apply_chat_template(
    messages,
    add_generation_prompt=True
)

# add the base64 encdoed image to the prompt
image_input = f"data:image/jpeg;base64,{imgb64}"
image_input = f"![]({image_input})"
prompt = prompt.replace("<image>", image_input)

In [ ]:
limerick = client.text_generation(
    prompt,
    model=vlm_model_id,
    max_new_tokens=200,
    seed=111
)
print(limerick)

### Generating speech from text

In [ ]:
tts_model_id = 'suno/bark'
speech_out = client.text_to_speech(
    text=limerick,
    model=tts_model_id
)

In [ ]:
from IPython.display import Audio

display(Audio(speech_out, rate=24000))
print(limerick)